<a href="https://colab.research.google.com/github/ksokoll/plant_desease_detection/blob/main/Anwendungsprojekt_smart_farming_final_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Initialization

In [ ]:
pip install anvil-uplink #Installation der Anvil-Uplink Funktion zur Kommunikation mit der Anvil-Web-App

In [ ]:
import torch #Nutzung des Modells mit der CPU/GPU
import cv2 #Open-Source Bibliothek zur Bildverarbeitung
from google.colab.patches import cv2_imshow #Google-Colab Patch zur Stabilisierung von CV2
import yaml #Notwendig zum Auslesen der YOLOv5 .yaml Datei
from collections import Counter #Bibliothek zur Nutzung eines Dictionaries zur Ermittlung der Klasse
import numpy as np #Python-Bibliothek zum Arbeiten mit Arrays
import base64 #Bibliothek zum Kodieren / Dekodieren von binären Daten für den Transport der Bilder

In [ ]:
import anvil.server #Import der Anvil-Server Funktion zur Verbindung mit Anvil

anvil.server.connect("<your anvil server>") #Das Anvil Server-Objekt. Den Code mit dem persönlichen Anvil-Code tauschen

In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

In [ ]:
device = torch.device('cpu') #Festlegung des Gerätes auf "CPU". Kann bei Bedarf auf "GPU" umgestellt werden.

In [ ]:
# Lade das Modell von Google Drive
model_path = '/content/drive/MyDrive/Colab Notebooks/Diseases_Colab_Files/best 300 epochs model L.pt'
model = torch.hub.load('/content/yolov5', 'custom', path=model_path, source='local')
model.to(device)
model = model.float()

# 2. File Upload and Prediction

In [ ]:
# Load and save class names from YAML file
data_yaml_path = '/content/drive/MyDrive/Colab Notebooks/Diseases_Dataset_filtered/data.yaml'
with open(data_yaml_path, 'r') as file:
    data_yaml = yaml.safe_load(file)
    class_names = data_yaml['names']

In [ ]:
@anvil.server.callable
def process_image(file):

  # Empfangen der Bilddaten von Anvil
  file_data = file.get_bytes()

  # Konvertieren der Bilddaten in eine Np-Array
  nparr = np.frombuffer(file_data, np.uint8)

  # Dekodieren des base64 bitstreams mithilfe von CV2
  img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)

  # Abfangen eines Fehlers, falls es kein Bild gibt
  if img is None:
        return "Fehler beim Einlesen des Bildes"

  # Anwenden des Machine-Learning Modells zur Vorhersage der Ergebnisse
  results = model(img, size=416)
  predictions = results.xyxy[0]

  # Anfangen eines Fehlers, falls keine Vorhersagen getroffen wurden
  if predictions is None or len(predictions) == 0:
        return "Keine Pflanzenkrankheiten gefunden"

  # Konvertieren der Vorhersagen in ein Dictionary, anschließend Majority-Voting zur Festlegung der Klasse
  predicted_class_indices = predictions[:, 5].int().tolist()
  predicted_classes = [class_names[i] for i in predicted_class_indices]
  predicted_class = Counter(predicted_classes).most_common(1)[0][0]

  # (Optional zu Demonstrationszwecken) Visuelles Kenntlichmachen der Vorhersagen auf dem Bild
  for det in predictions:
    x1, y1, x2, y2, conf, cls = det[:6]
    label = f"{class_names[int(cls)]} {conf:.2f}"
    cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
    cv2.putText(img, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

  # (Optional) Rückgabe des Bildes, einschließlich visueller Merkmale
  _, buffer = cv2.imencode('.jpg', img)
  img_bytes = buffer.tobytes()
  img_base64 = base64.b64encode(img_bytes).decode('utf-8')

  # Rückgabe der ermittelten Klasse, Rückgabe des Bildes mit visuellen Merkmalen
  return predicted_class, img_base64

In [ ]:
anvil.server.wait_forever()